# NFL Fantasy Prediction Engine

Complete training, prediction, and betting analysis notebook.

## Features
- **Train** ML models (Ridge, ElasticNet, XGBoost, LightGBM)
- **Predict** fantasy points for any week
- **Analyze** player props and betting lines
- **Save** predictions to database automatically

---

## 1. Setup Environment

In [ ]:
# Install dependencies
!pip install -q scikit-learn xgboost lightgbm pandas numpy matplotlib seaborn

In [ ]:
# Clone repository
!git clone https://github.com/netprocesssolutions/nfl_prediction_engine.git 2>/dev/null || echo "Repo exists"
%cd nfl_prediction_engine
!git pull
!git lfs pull

In [ ]:
# Import modules
import sys
sys.path.insert(0, '/content/nfl_prediction_engine')

from colab import setup, data, train, predict, betting, save

print("Modules loaded successfully!")

### Option A: Use database from GitHub LFS

In [ ]:
# Set database path (use LFS file from repo)
setup.set_db_path('/content/nfl_prediction_engine/phase_1/database/nfl_data (1).db')

# Verify database
setup.verify_database()

### Option B: Use database from Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set database path from Drive
setup.set_db_path('/content/drive/MyDrive/NFL/nfl_data.db')
setup.verify_database()

---
## 2. Explore Data

In [ ]:
# View available seasons
seasons = data.get_seasons()
print(f"Available seasons: {seasons}")

# View weeks in current season
weeks = data.get_weeks(2024)
print(f"2024 weeks: {weeks}")

In [ ]:
# View players for a specific week
players = data.get_players(season=2024, week=14, position='WR')
print(f"\nWRs in Week 14: {len(players)}")
players.head(10)

In [ ]:
# View player history
data.get_player_stats(player_name="Justin Jefferson", season=2024, limit=10)

---
## 3. Train Models

In [ ]:
# Train models (this will take a few minutes)
trainer = train.train_models(
    train_seasons=[2021, 2022, 2023],  # Training data
    val_season=2023,                    # Validation (early stopping)
    test_season=2024,                   # Test evaluation
    positions=None,                     # All positions (or ['QB', 'RB', 'WR', 'TE'])
    model_types=None,                   # All available models
    verbose=True
)

In [ ]:
# Quick training (for testing)
# trainer = train.train_models([2023], test_season=2024, model_types=['ridge', 'elasticnet'])

In [ ]:
# View saved models
train.list_saved_models()

---
## 4. Generate Predictions

### 4.1 Weekly Predictions

In [ ]:
# Predict for Week 14
SEASON = 2024
WEEK = 14

predictions = predict.predict_week(SEASON, WEEK, scoring='ppr')

In [ ]:
# Top 20 overall
predictions[['player_name', 'position', 'team', 'opponent', 'pred_fp_ppr']].head(20)

### 4.2 Position Rankings

In [ ]:
# Top QBs
qbs = predict.predict_position('QB', SEASON, WEEK, top_n=15)

In [ ]:
# Top RBs
rbs = predict.predict_position('RB', SEASON, WEEK, top_n=20)

In [ ]:
# Top WRs
wrs = predict.predict_position('WR', SEASON, WEEK, top_n=25)

In [ ]:
# Top TEs
tes = predict.predict_position('TE', SEASON, WEEK, top_n=15)

### 4.3 Specific Player Predictions

In [ ]:
# Single player prediction
predict.predict_player("CeeDee Lamb", SEASON, WEEK)

In [ ]:
# Compare multiple players
predict.compare_players(
    ["CeeDee Lamb", "Ja'Marr Chase", "Justin Jefferson", "Tyreek Hill"],
    SEASON, WEEK
)

### 4.4 Stat-Specific Predictions

In [ ]:
# Top receiving yards
predict.predict_stat('rec_yards', SEASON, WEEK, positions=['WR', 'TE'])

In [ ]:
# Top rushing yards
predict.predict_stat('rush_yards', SEASON, WEEK, positions=['RB'])

In [ ]:
# Players likely to score TDs (>0.5 expected)
predict.predict_stat('rec_tds', SEASON, WEEK, threshold=0.5)

In [ ]:
# Find sleepers
predict.get_sleepers(SEASON, WEEK, min_fp=8.0)

---
## 5. Betting & Props Analysis

### 5.1 Prop Sheets

In [ ]:
# Create full prop sheet
sheets = betting.create_prop_sheet(SEASON, WEEK)

In [ ]:
# Receiving props analysis
betting.analyze_receiving_props(SEASON, WEEK, top_n=25)

In [ ]:
# Rushing props analysis
betting.analyze_rushing_props(SEASON, WEEK, top_n=20)

In [ ]:
# Passing props analysis
betting.analyze_passing_props(SEASON, WEEK)

### 5.2 Analyze Specific Props

In [ ]:
# Analyze a single prop
betting.analyze_prop("CeeDee Lamb", "receiving_yards", 85.5, SEASON, WEEK)

In [ ]:
# Analyze passing prop
betting.analyze_prop("Josh Allen", "passing_yards", 250.5, SEASON, WEEK)

In [ ]:
# Analyze rushing prop
betting.analyze_prop("Saquon Barkley", "rushing_yards", 85.5, SEASON, WEEK)

### 5.3 Find Value Props

In [ ]:
# Create a DataFrame of props to analyze
import pandas as pd

# Enter your prop lines here
my_props = pd.DataFrame({
    'player_name': ['CeeDee Lamb', 'Ja\'Marr Chase', 'Justin Jefferson', 'Josh Allen', 'Saquon Barkley'],
    'prop_type': ['receiving_yards', 'receiving_yards', 'receiving_yards', 'passing_yards', 'rushing_yards'],
    'line': [85.5, 75.5, 80.5, 250.5, 75.5]
})

# Find value (>10% edge)
betting.find_value_props(SEASON, WEEK, my_props, min_edge_pct=10)

### 5.4 Anytime TD Scorers

In [ ]:
# Best TD scorer candidates
betting.anytime_td_scorers(SEASON, WEEK, min_td_prob=0.4)

---
## 6. Save Predictions to Database

In [ ]:
# Save ML predictions to database
save.save_ml_predictions(predictions, model_version="v1.0")

In [ ]:
# View saved predictions
save.list_saved_predictions()

In [ ]:
# Retrieve saved predictions
saved = save.get_saved_predictions('ml', season=2024, week=14)
saved.head()

### 6.1 Export Options

In [ ]:
# Export to CSV
save.export_predictions_csv(predictions, season=SEASON, week=WEEK)

In [ ]:
# Export to Google Drive
# save.export_to_drive(predictions, season=SEASON, week=WEEK)

In [ ]:
# Download predictions directly
save.download_predictions(predictions)

In [ ]:
# Backup database to Drive
# save.sync_database_to_drive()

---
## 7. Quick Reference

### Training
```python
trainer = train.train_models([2021, 2022, 2023], test_season=2024)
```

### Predictions
```python
predict.predict_week(2024, 14)                          # Full week
predict.predict_position('WR', 2024, 14)                # Position rankings
predict.predict_player('CeeDee Lamb', 2024, 14)         # Single player
predict.predict_stat('rec_yards', 2024, 14)             # Specific stat
predict.compare_players(['Player1', 'Player2'], 2024, 14)  # Compare
```

### Betting
```python
betting.analyze_prop('Player', 'receiving_yards', 85.5, 2024, 14)
betting.analyze_receiving_props(2024, 14)
betting.anytime_td_scorers(2024, 14)
```

### Saving
```python
save.save_ml_predictions(preds, model_version='v1.0')
save.export_predictions_csv(preds, season=2024, week=14)
save.download_predictions(preds)
```